In [249]:
import os
import pandas as pd
import numpy as np
import gensim
from nltk.tokenize import word_tokenize
import matplotlib.pyplot as plt
from collections import defaultdict
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import KFold
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from scipy.stats import spearmanr
%matplotlib inline  
import re

In [3]:
files = os.listdir('DATA')

In [4]:
files

['dialogs_2.csv', 'dialogs_1.csv', 'email_reports.csv']

In [4]:
dialog = pd.read_csv('DATA/email_reports.csv')

In [362]:
dialog

,SENDER_ID,SENDER_EMAIL,OPERATOR_ID,ACCEPTOR,SUBJECT,To.queue.time,agent.receive.time,treatment.end.time,CONTENT,ANSWERTEXT
0,token_7747,xxxx@yandex.ru,token_48,spb.vopros@tele2.ru,Обратная связь,2088-06-17 02:31:32,2088-06-18 19:48:26,2088-06-18 20:24:59,Тело письма\r\n\r\nMSISDN: +xxx xxx xx xx\r\nО...,"\r\n\r\nЗдравствуйте, xxxxx xxxxx xxxxx!\r\n\r..."
1,token_14226,xxxx@mail.ru,token_50,Saransk.Vopros@tele2.ru,Re[2]: Обратная связь,2088-06-17 02:31:52,2088-06-18 18:43:45,2088-06-18 18:58:43,\r\nвся необходимая информация отображена в ...,"Здравствуйте, xxxxx xxxxx xxxxx!\r\nБлагодарим..."
2,token_23942,xxxx@mail.ru,token_3,rostov.vopros@tele2.ru,Обратная связь,2088-06-17 02:33:14,2088-06-18 18:43:45,2088-06-18 18:47:06,Тело письма\r\n\r\nMSISDN: +xxx xxx xx xx\r\nО...,"\r\n\r\nЗдравствуйте, xxxxx xxxxx xxxxx!\r\nБл..."
3,token_18368,xxxx@mail.ru,token_53,krasnodar.vopros@tele2.ru,Обратная связь,2088-06-17 02:33:36,2088-06-18 19:49:10,2088-06-18 19:53:57,Тело письма\r\n\r\nMSISDN: +xxx xxx xx xx\r\nО...,"\r\n\r\nЗдравствуйте, xxxxx xxxxx xxxxx! ¦\r\n..."
4,token_21039,xxxx@mail.ru,token_34,kostroma.vopros@tele2.ru,Обратная связь,2088-06-17 02:33:56,2088-06-18 19:49:17,2088-06-18 19:59:48,"Тело письма\r\n\r\nОтметьте, в каком регионе В...","\r\n\r\nАлександр, доброго времени суток! ;)\r..."
5,token_24778,xxxx@mal.ru,token_103,Moscow.Vopros@tele2.ru,Обратная связь,2088-06-17 02:37:23,2088-06-18 19:52:46,2088-06-18 20:03:32,"Тело письма\r\n\r\nОтметьте, в каком регионе В...","\r\n\r\nЗдравствуйте, xxxxx xxxxx xxxxx!\r\nСп..."
6,token_12422,xxxx@gmail.com,token_69,nnov.vopros@tele2.ru,Обратная связь,2088-06-17 02:37:36,2088-06-18 20:53:39,2088-06-18 20:57:10,"Тело письма\r\n\r\nОтметьте, в каком регионе В...","Здравствуйте, xxxxx xxxxx xxxxx!\r\nБлагодарим..."
7,token_24424,xxxx@mail.ru,token_40,vladimir.vopros@tele2.ru,"Заполнена форма ""Ск<U+FFFD><U+FFFD>рая абонент...",2088-06-17 02:37:56,2088-06-18 20:55:16,2088-06-18 20:59:46,"Заполнена форма ""Скорая абонентская""\r\n\r\nms...","\r\n\r\nПриветствую Вас, xxxxx xxxxx xxxxx!\r\..."
8,token_19238,xxxx@gmail.com,token_28,Perm.Vopros@tele2.ru,Обратная связь,2088-06-17 02:38:02,2088-06-18 18:44:19,2088-06-18 18:51:39,Тело письма\r\n\r\nMSISDN: +xxx xxx xx xx\r\nО...,"Здравствуйте, Василий Владимирович!\r\nСпасибо..."
9,token_19252,xxxx@mail.ru,token_69,Moscow.Vopros@tele2.ru,Обратная связь,2088-06-17 02:38:18,2088-06-18 20:57:11,2088-06-18 21:10:10,Тело письма\r\n\r\nMSISDN: xxx xxx xx xx\r\nО...,"Здравствуйте, xxxxx xxxxx xxxxx!\r\nБлагодарим..."


In [113]:
dialog[dialog['SENDER_ID']=='token_23942']['CONTENT'].apply(preprocess_raw).values

array(['Интернет опять еле тянет. Преимущественно сеть «Е» и иногда мелькает 3g. В чем опять проблема? Уже несколько дней так\r',
       'Остался долг при переходе с МТС на Теле2, вчера заходил в салон МТС, долг оплатил, а номер заново не заработал, почему? \r'],
      dtype=object)

In [292]:
dialog['CONTENT'].iloc[0]

'Тело письма\r\n\r\nMSISDN: +xxx xxx xx xx\r\nОтметьте, в каком регионе Вы постоянно проживаете:: Санкт-Петербург и Ленинградская область\r\nКак к вам обращаться: xxxxx xxxxx xxxxx\r\nКонтактный номер телефона: +xxx xxx xx xx\r\nКонтактный email: xxx@yandex.ru\r\nВыберите тему обращения: прочее\r\nЗадайте ваш вопрос или опишите проблему: Здравствуйте. Я и моя жена xxxxx xxxxx xxxxx, являемся абонентами теле2 уже около 10лет! Сим-карты были нам подарены братом жены, на данный момент проживающим в другой области, встал вопрос о замене сим-карт, так как они устаревшие и не поддерживают высокоскоростной интернет. Обратились в офис теле2 по поводу замены сим-карт заполнили анкету и заявление, а так же их зарегестрировали, через 2 дня пришёл отказ. Хотел бы поинтересоваться по чему отказали в замене? Не хотелось бы менять оператора из-за данного инцидента, но видимо придётся!!!!Спасибо. \r\nВыражаю согласие* на обработку персональных данных в соответствии со ст.9 ФЗ №152 "О персональных данн

In [360]:
dialog['SENDER_ID'].value_counts()

token_37216    5795
token_22649    2797
token_16970    2038
token_12956    1799
token_7316      878
token_37217     825
token_37218     794
token_22650     712
token_6142      583
token_12843     555
token_22651     450
token_16972     447
token_NA        364
token_16973     293
token_7255      205
token_6079      174
token_37224     173
token_37219     143
token_22652     137
token_16974      69
token_15848      63
token_22655      61
token_16975      56
token_14734      48
token_31921      45
token_5429       43
token_24431      41
token_37232      40
token_37220      37
token_6518       34
               ... 
token_42053       1
token_32768       1
token_32402       1
token_31172       1
token_37223       1
token_41553       1
token_33215       1
token_36141       1
token_33783       1
token_35866       1
token_27377       1
token_44269       1
token_28913       1
token_31016       1
token_34343       1
token_28628       1
token_35926       1
token_41932       1
token_30041       1


In [361]:
dialog[dialog['SENDER_ID'] == 'token_6518']

,SENDER_ID,SENDER_EMAIL,OPERATOR_ID,ACCEPTOR,SUBJECT,To.queue.time,agent.receive.time,treatment.end.time,CONTENT,ANSWERTEXT
6702,token_6518,xxxx@yandex.ru,token_114,Moscow.Vopros@tele2.ru,Обратная связь,2088-06-20 01:50:00,2088-06-21 02:12:37,2088-06-21 02:18:54,Тело письма\r\n\r\nMSISDN: +xxx xxx xx xx\r\nО...,"\r\n\r\nЗдравствуйте, xxxxx xxxxx xxxxx!\r\nБл..."
10485,token_6518,xxxx@yandex.ru,token_105,Moscow.Vopros@tele2.ru,Re: Обратная связь,2088-06-21 17:42:17,2088-06-22 17:29:58,2088-06-22 17:29:58,"Спасибо\r\n\r\n_xxxx xxx xx xx, 23:19, ""xxx@te...",Не требуется ответа
45857,token_6518,xxxx@gmail.com,token_113,Yoshkar-Ola.Vopros@tele2.ru,Обратная связь,2088-07-06 09:14:47,2088-07-07 21:09:25,2088-07-07 22:27:10,Тело письма\r\n\r\nMSISDN: +xxx xxx xx xx\r\nО...,"\r\n\r\nЗдравствуйте, xxxxx xxxxx xxxxx!\r\nСп..."
49634,token_6518,xxxx@gmail.com,token_117,Yoshkar-Ola.Vopros@tele2.ru,Re: Обратная связь,2088-07-07 23:06:10,2088-07-20 00:12:22,2088-07-20 00:12:22,"\r\n\r\nТариф изменили без моего согласия, я н...",переведено
49635,token_6518,xxxx@gmail.com,token_10,Yoshkar-Ola.Vopros@tele2.ru,Re: Обратная связь,2088-07-07 23:06:10,2088-07-21 16:16:36,2088-07-21 16:16:36,"\r\n\r\nТариф изменили без моего согласия, я н...",переведено
49636,token_6518,xxxx@gmail.com,token_81,Yoshkar-Ola.Vopros@tele2.ru,Re: Обратная связь,2088-07-07 23:06:10,2088-07-12 21:12:41,2088-07-12 21:12:42,"\r\n\r\nТариф изменили без моего согласия, я н...",переведено
49637,token_6518,xxxx@gmail.com,token_10,Yoshkar-Ola.Vopros@tele2.ru,Re: Обратная связь,2088-07-07 23:06:10,2088-07-21 17:25:32,2088-07-21 17:25:32,"\r\n\r\nТариф изменили без моего согласия, я н...",переведено
49638,token_6518,xxxx@gmail.com,token_117,Yoshkar-Ola.Vopros@tele2.ru,Re: Обратная связь,2088-07-07 23:06:10,2088-07-20 00:23:11,2088-07-20 00:23:11,"\r\n\r\nТариф изменили без моего согласия, я н...",переведено
49639,token_6518,xxxx@gmail.com,token_117,Yoshkar-Ola.Vopros@tele2.ru,Re: Обратная связь,2088-07-07 23:06:10,2088-07-13 17:52:42,2088-07-13 17:52:42,"\r\n\r\nТариф изменили без моего согласия, я н...",переведено
49640,token_6518,xxxx@gmail.com,token_103,Yoshkar-Ola.Vopros@tele2.ru,Re: Обратная связь,2088-07-07 23:06:10,2088-07-13 21:34:20,2088-07-13 21:34:20,"\r\n\r\nТариф изменили без моего согласия, я н...",переведено


In [363]:

tm = dialog[['To.queue.time']].apply(pd.to_datetime)

In [367]:
tm.dropna().min()

To.queue.time   2088-06-17 02:31:32
dtype: datetime64[ns]

In [48]:
diffs = tm['treatment.end.time'] - tm['agent.receive.time']

In [317]:
data = pd.DataFrame({'y':diffs.apply(lambda x: x.seconds), 'text':dialog['CONTENT'], 
                     'user':dialog['SENDER_ID'], 'answer': dialog['ANSWERTEXT']})

In [320]:
data.dropna(inplace=True)

In [321]:
data['prep_text'] = data['text'].apply(preprocess)

In [326]:
data['prep_text'].values

array(['здравствуйте моя жена xxxxx xxxxx xxxxx являемся абонентами теле2 уже около 10лет были нам подарены братом жены данный момент проживающим другой области встал вопрос замене так как они устаревшие поддерживают высокоскоростной интернет обратились офис теле2 поводу замены заполнили анкету заявление так зарегестрировали через дня пришёл отказ хотел поинтересоваться чему отказали замене хотелось менять оператора данного инцидента видимо придётся спасибо',
       'интернет опять еле тянет преимущественно сеть иногда мелькает чем опять проблема уже несколько дней так',
       'номера 312 приходят странные смс уведомлении списании', ...,
       'роутер xxx подключаю модем билайна работает подключаю модем теле2 симкой xxx xxx роутер подключается xxx xxx открываются диагностика выдает отвечает что делать ответить можно xxx',
       'здравствуйте какой меня тариф',
       'добрый вечер после обновлений подрят так появилось вход систему отпечатку пальца моём устройстве iphone ios beta'],


In [325]:
data = data[(data.y > 0) & (data.y < 15000)]

In [324]:
data = data[data['prep_text'] != '']

In [339]:
data['len_ans'] = data['answer'].apply(len)

In [328]:
data['len_q'] = data['text'].apply(len)

In [329]:
data['len_ans'].values

array([1784, 2003, 1342, ..., 1137, 1176, 1117])

In [340]:
spearmanr(data['len_ans'], data['y'])

SpearmanrResult(correlation=0.10377875361495113, pvalue=1.4669372799093195e-228)

In [335]:
tfidf = TfidfVectorizer(max_features=25000, min_df=5, max_df=0.3)
vecs = tfidf.fit_transform(data['prep_text'].values)

In [332]:
y = data['y'].values

In [84]:
ft = gensim.models.FastText.load('fast_text')

In [336]:
texts_raw = data['text'].apply(preprocess_raw).values

In [337]:
def preprocess_ans(text):
    sents = [sent for sent in text.split('\r\n') if sent][1:-3]
    texts = ' '.join(sents)
    
    return texts

In [338]:
texts_ans = data['answer'].apply(preprocess_ans).values

In [344]:
texts = data['prep_text'].values
vecs_ft = np.zeros((len(texts), 200))

for i, sent in enumerate(texts):
    words = sent.split()
    v = np.zeros((len(words), 200))
    weights = []
    for j, word in enumerate(words):
        try:
            v[j] = ft[word]
            weights.append(1)
        except KeyError:
            weights.append(0)
            continue
    if not v.any():
        continue
    v = np.average(v, axis=0, weights=weights)
    vecs_ft[i] = v

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.


In [346]:
import gc
gc.collect()

110

In [341]:
scores = []
i = 0
kf = KFold(n_splits=10)
regr = Lasso(0.1)
for train, valid in kf.split(vecs):
    print('Doing fold ', i)
    X_train, X_valid = vecs[train], vecs[valid]
    y_train, y_valid = y[train], y[valid]
    
    regr.fit(X_train, y_train)
    pred = regr.predict(X_valid)
    
    score = mean_absolute_error(y_valid, pred)
    scores.append(score)
    i += 1

print('Score - ', np.mean(scores))

Doing fold  0
Doing fold  1
Doing fold  2
Doing fold  3
Doing fold  4
Doing fold  5
Doing fold  6
Doing fold  7
Doing fold  8
Doing fold  9
Score -  691.7031735200526


In [342]:
features = tfidf.get_feature_names()
coef = regr.coef_

fc = list(zip(features, coef))

In [343]:
[x for x in fc if x[1] > 0.1 or  x[1] < -0.1]

[('150', 567.093074703973),
 ('200', 45.50692439903271),
 ('270', 68.52415374526879),
 ('300', 6.67363142761298),
 ('315', 234.40705388567903),
 ('350', 35.40667189903148),
 ('400', 106.15987371437187),
 ('448', 345.24879697156115),
 ('460', 1517.0434777829314),
 ('500', 134.06762477347993),
 ('50р', 258.6368618471909),
 ('550', 1035.7710900059237),
 ('800', 206.68669004497997),
 ('facetime', 413.02578457277906),
 ('sms', -135.11840941600346),
 ('tele2', 127.96936921311419),
 ('xxx', 112.9057502114318),
 ('xxxxx', 71.72822075553867),
 ('абон', 151.30776521187718),
 ('абонент', 185.90071926773095),
 ('абонента', -14.757906656698449),
 ('абоненту', -63.134294690994714),
 ('автоплатеж', 12.670019932678892),
 ('адресу', -177.91599265207492),
 ('баланса', 61.26056478261688),
 ('балансе', 327.1000160619558),
 ('банк', 57.24380953282129),
 ('безлимит', 319.2148223574833),
 ('бесплатных', 83.52986310079571),
 ('беспредельно', -102.33480171637271),
 ('благодарю', -1.0467744955382203),
 ('больше

In [350]:
scores = []
i = 0
kf = KFold(n_splits=10)
# regr = Ridge(1009)

params = {'n_estimators':2000,
             'max_depth':6,
#              'lambda':100,
             'eta': 0.1, 
            "colsample_bytree":0.9,
#             "sample":0.7,
            "min_child_weight":100,
             "tweedie_variance_power":1.1,
             'objective': 'reg:tweedie',
#               'tree_method':'gpu_hist',
             'eval_metric':'mae',
         }

for train, valid in kf.split(vecs_ft):
    print('Doing fold ', i)
    
    X_train, X_valid = vecs_ft[train], vecs_ft[valid]
    y_train, y_valid = y[train], y[valid]
    
    xgb_train = xgb.DMatrix(X_train, y_train)
    xgb_val = xgb.DMatrix(X_valid, y_valid)
    
    model = xgb.train(params, xgb_train, 5000, 
                      evals=[(xgb_train, 'train'), (xgb_val, 'val')], 
                      early_stopping_rounds=10,verbose_eval=30)
    
    pred = model.predict(xgb_val)
    
    
    score = mean_absolute_error(y_valid, pred)
    scores.append(score)
    i += 1

print('Score - ', np.mean(scores))

Doing fold  0
[0]	train-mae:828.42	val-mae:723.352
Multiple eval metrics have been passed: 'val-mae' will be used for early stopping.

Will train until val-mae hasn't improved in 10 rounds.
Stopping. Best iteration:
[10]	train-mae:561.624	val-mae:487.643

Doing fold  1
[0]	train-mae:823.541	val-mae:767.258
Multiple eval metrics have been passed: 'val-mae' will be used for early stopping.

Will train until val-mae hasn't improved in 10 rounds.
Stopping. Best iteration:
[10]	train-mae:557.799	val-mae:526.845

Doing fold  2
[0]	train-mae:814.42	val-mae:849.357
Multiple eval metrics have been passed: 'val-mae' will be used for early stopping.

Will train until val-mae hasn't improved in 10 rounds.
Stopping. Best iteration:
[11]	train-mae:546.452	val-mae:599.913

Doing fold  3
[0]	train-mae:809.95	val-mae:889.591
Multiple eval metrics have been passed: 'val-mae' will be used for early stopping.

Will train until val-mae hasn't improved in 10 rounds.
Stopping. Best iteration:
[11]	train-mae:

In [351]:
km = MiniBatchKMeans(2000, verbose=1)
km.fit(vecs_ft)

Init 1/3 with method: k-means++


/usr/local/lib/python3.5/dist-packages/sklearn/cluster/k_means_.py:1418: RuntimeWarning: init_size=300 should be larger than k=2000. Setting it to 3*k
  init_size=init_size)


Inertia for init 1/3: 2171.961784
Init 2/3 with method: k-means++
Inertia for init 2/3: 1877.667968
Init 3/3 with method: k-means++
Inertia for init 3/3: 2321.576778
Minibatch iteration 1/96300: mean batch inertia: 44.441407, ewa inertia: 44.441407 
Minibatch iteration 2/96300: mean batch inertia: 43.873320, ewa inertia: 44.440226 
Minibatch iteration 3/96300: mean batch inertia: 42.306473, ewa inertia: 44.435791 
Minibatch iteration 4/96300: mean batch inertia: 43.266107, ewa inertia: 44.433359 
Minibatch iteration 5/96300: mean batch inertia: 41.853788, ewa inertia: 44.427997 
Minibatch iteration 6/96300: mean batch inertia: 42.387212, ewa inertia: 44.423755 
Minibatch iteration 7/96300: mean batch inertia: 37.598429, ewa inertia: 44.409567 
Minibatch iteration 8/96300: mean batch inertia: 41.751080, ewa inertia: 44.404041 
Minibatch iteration 9/96300: mean batch inertia: 39.625047, ewa inertia: 44.394107 
[MiniBatchKMeans] Reassigning 50 cluster centers.
Minibatch iteration 10/96300

Minibatch iteration 92/96300: mean batch inertia: 38.405297, ewa inertia: 43.700100 
Minibatch iteration 93/96300: mean batch inertia: 38.345481, ewa inertia: 43.688969 
Minibatch iteration 94/96300: mean batch inertia: 37.159352, ewa inertia: 43.675396 
Minibatch iteration 95/96300: mean batch inertia: 37.460139, ewa inertia: 43.662477 
Minibatch iteration 96/96300: mean batch inertia: 33.955091, ewa inertia: 43.642298 
Minibatch iteration 97/96300: mean batch inertia: 40.155293, ewa inertia: 43.635050 
Minibatch iteration 98/96300: mean batch inertia: 40.937296, ewa inertia: 43.629442 
Minibatch iteration 99/96300: mean batch inertia: 35.995281, ewa inertia: 43.613573 
[MiniBatchKMeans] Reassigning 50 cluster centers.
Minibatch iteration 100/96300: mean batch inertia: 37.085849, ewa inertia: 43.600003 
Minibatch iteration 101/96300: mean batch inertia: 34.267831, ewa inertia: 43.580605 
Minibatch iteration 102/96300: mean batch inertia: 37.651751, ewa inertia: 43.568280 
Minibatch it

Minibatch iteration 188/96300: mean batch inertia: 39.742342, ewa inertia: 42.438113 
Minibatch iteration 189/96300: mean batch inertia: 38.703362, ewa inertia: 42.430349 
[MiniBatchKMeans] Reassigning 50 cluster centers.
Minibatch iteration 190/96300: mean batch inertia: 39.828872, ewa inertia: 42.424941 
Minibatch iteration 191/96300: mean batch inertia: 33.933058, ewa inertia: 42.407289 
Minibatch iteration 192/96300: mean batch inertia: 36.232881, ewa inertia: 42.394455 
Minibatch iteration 193/96300: mean batch inertia: 35.087425, ewa inertia: 42.379265 
Minibatch iteration 194/96300: mean batch inertia: 35.460406, ewa inertia: 42.364883 
Minibatch iteration 195/96300: mean batch inertia: 35.058586, ewa inertia: 42.349696 
Minibatch iteration 196/96300: mean batch inertia: 33.754928, ewa inertia: 42.331830 
Minibatch iteration 197/96300: mean batch inertia: 35.641268, ewa inertia: 42.317922 
Minibatch iteration 198/96300: mean batch inertia: 35.633778, ewa inertia: 42.304028 
Mini

...

Minibatch iteration 1146/96300: mean batch inertia: 31.012701, ewa inertia: 34.289220 
Minibatch iteration 1147/96300: mean batch inertia: 36.623784, ewa inertia: 34.294072 
Minibatch iteration 1148/96300: mean batch inertia: 28.240589, ewa inertia: 34.281489 
Minibatch iteration 1149/96300: mean batch inertia: 31.690925, ewa inertia: 34.276104 
Minibatch iteration 1150/96300: mean batch inertia: 31.098635, ewa inertia: 34.269499 
Minibatch iteration 1151/96300: mean batch inertia: 32.715440, ewa inertia: 34.266269 
[MiniBatchKMeans] Reassigning 50 cluster centers.
Minibatch iteration 1152/96300: mean batch inertia: 31.963818, ewa inertia: 34.261483 
Minibatch iteration 1153/96300: mean batch inertia: 31.127475, ewa inertia: 34.254968 
Minibatch iteration 1154/96300: mean batch inertia: 34.770777, ewa inertia: 34.256040 
Minibatch iteration 1155/96300: mean batch inertia: 30.745164, ewa inertia: 34.248742 
Minibatch iteration 1156/96300: mean batch inertia: 30.331675, ewa inertia: 34.2

Minibatch iteration 1237/96300: mean batch inertia: 33.060931, ewa inertia: 33.914010 
Minibatch iteration 1238/96300: mean batch inertia: 30.377606, ewa inertia: 33.906659 
Minibatch iteration 1239/96300: mean batch inertia: 31.045648, ewa inertia: 33.900712 
Minibatch iteration 1240/96300: mean batch inertia: 35.017440, ewa inertia: 33.903033 
Minibatch iteration 1241/96300: mean batch inertia: 31.600629, ewa inertia: 33.898247 
Minibatch iteration 1242/96300: mean batch inertia: 32.355089, ewa inertia: 33.895039 
Minibatch iteration 1243/96300: mean batch inertia: 31.337015, ewa inertia: 33.889722 
Minibatch iteration 1244/96300: mean batch inertia: 35.661325, ewa inertia: 33.893405 
Minibatch iteration 1245/96300: mean batch inertia: 37.541610, ewa inertia: 33.900988 
Minibatch iteration 1246/96300: mean batch inertia: 33.756102, ewa inertia: 33.900687 
Minibatch iteration 1247/96300: mean batch inertia: 30.846457, ewa inertia: 33.894338 
[MiniBatchKMeans] Reassigning 50 cluster ce

Minibatch iteration 1328/96300: mean batch inertia: 31.818729, ewa inertia: 33.652770 
Minibatch iteration 1329/96300: mean batch inertia: 32.601519, ewa inertia: 33.650585 
Minibatch iteration 1330/96300: mean batch inertia: 36.284128, ewa inertia: 33.656059 
Minibatch iteration 1331/96300: mean batch inertia: 34.550015, ewa inertia: 33.657918 
[MiniBatchKMeans] Reassigning 50 cluster centers.
Minibatch iteration 1332/96300: mean batch inertia: 32.481673, ewa inertia: 33.655473 
Minibatch iteration 1333/96300: mean batch inertia: 31.353132, ewa inertia: 33.650687 
Minibatch iteration 1334/96300: mean batch inertia: 30.297428, ewa inertia: 33.643716 
Minibatch iteration 1335/96300: mean batch inertia: 32.589292, ewa inertia: 33.641524 
Minibatch iteration 1336/96300: mean batch inertia: 35.168511, ewa inertia: 33.644699 
Minibatch iteration 1337/96300: mean batch inertia: 32.893406, ewa inertia: 33.643137 
Minibatch iteration 1338/96300: mean batch inertia: 33.197140, ewa inertia: 33.6

Minibatch iteration 1421/96300: mean batch inertia: 33.165437, ewa inertia: 33.421820 
Converged (lack of improvement in inertia) at iteration 1421/96300
Computing label assignment and total inertia


MiniBatchKMeans(batch_size=100, compute_labels=True, init='k-means++',
        init_size=None, max_iter=100, max_no_improvement=10,
        n_clusters=2000, n_init=3, random_state=None,
        reassignment_ratio=0.01, tol=0.0, verbose=1)

In [220]:
texts_raw

array(['Здравствуйте. Я и моя жена xxxxx xxxxx xxxxx, являемся абонентами теле2 уже около 10лет! Сим-карты были нам подарены братом жены, на данный момент проживающим в другой области, встал вопрос о замене сим-карт, так как они устаревшие и не поддерживают высокоскоростной интернет. Обратились в офис теле2 по поводу замены сим-карт заполнили анкету и заявление, а так же их зарегестрировали, через 2 дня пришёл отказ. Хотел бы поинтересоваться по чему отказали в замене? Не хотелось бы менять оператора из-за данного инцидента, но видимо придётся!!!!Спасибо. \r',
       'Добрый вечер.\r',
       'Интернет опять еле тянет. Преимущественно сеть «Е» и иногда мелькает 3g. В чем опять проблема? Уже несколько дней так\r',
       ..., '',
       'Добрый вечер!!  После 6 обновлений подрят!!!!!!,так и не появилось,вход в систему по отпечатку пальца,на моём устройстве!!  iPhone 6s,IOS 12 beta 7. \r',
       ''], dtype=object)

In [352]:
labels = km.labels_
cls = defaultdict(list)

for i, l in enumerate(labels):
    cls[l].append(texts_raw[i])

In [353]:
cls_y = defaultdict(list)

for i, l in enumerate(labels):
    cls_y[l].append(y[i])

In [354]:
cls_mean_std = {}
for cl in cls_y:
    mean = np.mean(cls_y[cl])
    std = np.std(cls_y[cl])
    cls_mean_std[cl] = (mean, std)

In [355]:
cls_ans = defaultdict(list)

for i, l in enumerate(labels):
    cls_ans[l].append(texts_ans[i])

In [356]:
cls_len = defaultdict(float)

for i, l in enumerate(labels):
    cls_len[l] += len(texts_ans[i])/len(cls[l])

In [212]:
c

{0: (806.8580164710005, 1366.0369607265102),
 1: (689.8333333333334, 485.1691858402478),
 2: (811.3626048347311, 1413.9361969552967),
 3: (806.8895058300944, 1338.0317070156698),
 4: (668.0, 0.0),
 5: (923.0, 556.0),
 6: (1030.857142857143, 2176.4812205487465),
 7: (675.0, 405.72034703721727),
 8: (6653.0, 0.0),
 9: (943.0, 1770.7384935387834),
 10: (490.9, 269.95571859103114),
 11: (375.3333333333333, 270.3518283849974),
 12: (612.5, 164.5),
 13: (504.54285714285714, 387.14901840115726),
 14: (162.0, 0.0),
 15: (460.8333333333333, 434.6015096870031),
 16: (504.6666666666667, 240.78252059113888),
 17: (443.0, 267.1191993598863),
 18: (817.5074626865671, 1507.2731964545371),
 19: (835.1, 1657.8834971131114),
 20: (368.5, 24.5),
 21: (702.564705882353, 877.5909330736895),
 22: (1679.75, 1663.6069810805675),
 23: (953.796875, 1612.9153552233404),
 24: (470.0, 0.0),
 25: (1889.6666666666667, 1143.722091341346),
 26: (666.5, 380.5),
 27: (932.1451612903226, 1339.534582712607),
 28: (810.421

In [163]:
f = open('кластеры/вопросы_время.txt', 'w')
for cl in cls:
    f.write('######Кластер --  ' + str(cl) + '\n')
    f.write('Среднее - ' + str(cls_mean_std[cl][0]) + '\n')
    f.write('Std - ' + str(cls_mean_std[cl][1]) + '\n')
    f.write('\n'.join([x for x in cls[cl]]))
    f.write('\n\n\n')
f.close()

In [357]:
[x for x in sorted(cls_mean_std.items(), key=lambda x: -x[1][0]) if len(cls[x[0]]) > 10]

[(1455, (2597.181818181818, 4176.229156421155)),
 (933, (2486.923076923077, 4428.301992021815)),
 (866, (2035.8181818181818, 2784.2686592608384)),
 (1484, (2023.6666666666667, 3748.014881981602)),
 (1953, (1990.3055555555557, 3489.1889125908497)),
 (262, (1986.5454545454545, 3152.2474627027145)),
 (540, (1946.65, 2618.4885960225224)),
 (1621, (1839.4615384615386, 2665.779107225636)),
 (1427, (1829.8636363636363, 3002.8349982740488)),
 (966, (1823.36, 2571.539832551695)),
 (900, (1821.4375, 2182.1089560546125)),
 (1421, (1812.6538461538462, 2904.101793383173)),
 (833, (1803.060606060606, 3091.5135837888774)),
 (988, (1782.35, 3385.6872016623156)),
 (774, (1749.125, 3728.4654273183673)),
 (273, (1736.3636363636363, 3985.1746459443516)),
 (1814, (1686.25, 3412.8005820000676)),
 (1367, (1667.5, 3030.0434402826636)),
 (1098, (1655.5454545454545, 4051.1992021136916)),
 (1527, (1577.0, 1976.3208241578593)),
 (214, (1551.3125, 2301.1625518080527)),
 (125, (1536.3181818181818, 2287.574149545632

In [358]:
cls[1455]

['Не предоставляются услуги Интернета,предоставляются ли мне услуги мой онлайн? \r',
 'Подключил тариф мой разговор а услуги заблокированые\r',
 'Почему тариф мой заблокирован , не могу воспользоваться интернетов в пакете услуг \r',
 'НЕ отключайте пакет услуг Минуту1  мой разговор\r',
 'Мой пакет услуг мобильного интернета ещё действует?\r',
 'Услуги интернета по тарифу мой разговор за пределами Нижегородской области?\r',
 'Подключите, пожалуйста, мой пакет услуг!!!\r',
 'Почему, не списался платеж 21.07.xxxxx и мне не подключили мой пакет услуг? \r',
 'Отключить дополнительную услугу 500 мг оставить только мой тариф\r',
 'Мой тарифный план "Мой-онлайн04 xxxxx" платных услуг нет почему не подключен мой пакет услуг. Куда ушли мои деньги?\r',
 'Отключить дополнительную услугу ,которая входит в мой тариф \r']

In [271]:
for x in cls_ans[26]:
    print(x)
    print('---------')

Спасибо, что Вы нам написали. Мне очень жаль, что Вам пришлось столкнуться с такой ситуацией. Приносим свои извинения за задержку в предоставлении ответа. По техническим причинам мы не смогли ответить Вам в обещанные сроки. Если я правильно поняла, Вы имеете в виду мобильное приложение "Мой Tele2". Действительно, сейчас могут быть трудности со входом в приложение. Но наши специалисты уже занимаются этим вопросом. Они делают все возможное, чтобы как можно быстрее восстановить работоспособность сервиса. Пока Вы можете зайти в личный кабинет с сайта Tele2 (https://orenburg.tele2.ru/lk). В web-версии личного кабинета Вы можете установить постоянный пароль (в разделе "Профиль и настройки" - "Безопасность и авторизация" - "Смена пароля").  И после этого заходить в мобильное приложение "Мой Tele2" можно будет, используя данный пароль. Мы приносим Вам искренние извинения за доставленные неудобства! В качестве извинения я зачислила на Ваш счет компенсацию в размере 150 руб. Можете проверить бал

In [265]:
cls_len[5]

1158.457142857143

In [106]:
def preprocess_raw(text):
    q = re.search('Задайте ваш вопрос или опишите проблему: (.*)', text)
    if not q: return ''
    
    text = q.group(1)
    return text

In [8]:
dialog['Тональность'].value_counts()

Нейтральный    156410
Негативный       3486
Позитивный       1332
Name: Тональность, dtype: int64

In [11]:
dialog['Категория.автора'].value_counts()

Обычный         455821
Жалобщик         66554
Лидер мнения     23591
Name: Категория.автора, dtype: int64

In [12]:
dialog['Тип.обращения'].value_counts()

Личное сообщение    320707
Комментарий         193946
Пост                 44923
Репост                  51
Name: Тип.обращения, dtype: int64

In [316]:
def preprocess(text):
    q = re.search('Задайте ваш вопрос или опишите проблему: (.+)\r\nВыраж', text, re.DOTALL)
    if not q: return ''
    
    text = q.group(1)
    tokens = [word.lower() for word in word_tokenize(text) if word.isalnum() and len(word) > 2 and len(word) < 20]
    return ' '.join(tokens)

In [315]:
re.findall('Задайте ваш вопрос или опишите проблему: (.+)\r\nВыраж', (dialog['CONTENT'].iloc[0]), re.DOTALL)

['Здравствуйте. Я и моя жена xxxxx xxxxx xxxxx, являемся абонентами теле2 уже около 10лет! Сим-карты были нам подарены братом жены, на данный момент проживающим в другой области, встал вопрос о замене сим-карт, так как они устаревшие и не поддерживают высокоскоростной интернет. Обратились в офис теле2 по поводу замены сим-карт заполнили анкету и заявление, а так же их зарегестрировали, через 2 дня пришёл отказ. Хотел бы поинтересоваться по чему отказали в замене? Не хотелось бы менять оператора из-за данного инцидента, но видимо придётся!!!!Спасибо. ']

In [319]:
preprocess(dialog['CONTENT'].iloc[0])

'здравствуйте моя жена xxxxx xxxxx xxxxx являемся абонентами теле2 уже около 10лет были нам подарены братом жены данный момент проживающим другой области встал вопрос замене так как они устаревшие поддерживают высокоскоростной интернет обратились офис теле2 поводу замены заполнили анкету заявление так зарегестрировали через дня пришёл отказ хотел поинтересоваться чему отказали замене хотелось менять оператора данного инцидента видимо придётся спасибо'

In [16]:
def preprocess(text):
    tokens = [word for word in word_tokenize(text) if word.isalnum() and len(word) > 2 and len(word) < 20]
    return ' '.join(tokens)

In [21]:
ft = gensim.models.FastText([x.lower().split() for x in texts.values], size=200)

In [26]:
w2v = gensim.models.Word2Vec([x.lower().split() for x in texts.values], size=200, max_vocab_size=100000, sg=1)

In [25]:
ft.most_similar('как')

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('xxкак', 0.7733761668205261),
 ('некак', 0.674433708190918),
 ('какбы', 0.6724148392677307),
 ('какпя', 0.6707828640937805),
 ('кааак', 0.6285825967788696),
 ('каааак', 0.6001545190811157),
 ('какао', 0.5837725400924683),
 ('какаха', 0.582423210144043),
 ('както', 0.5822722315788269),
 ('кактус', 0.5809881091117859)]

In [30]:
w2v.most_similar('ловит')

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('ловил', 0.7401947379112244),
 ('неловит', 0.7152345180511475),
 ('берет', 0.714731752872467),
 ('жукова', 0.7143168449401855),
 ('бункере', 0.7141497135162354),
 ('ловился', 0.7068375945091248),
 ('слабенько', 0.6991263031959534),
 ('ловят', 0.6978026032447815),
 ('гонит', 0.6964523792266846),
 ('толчке', 0.6962436437606812)]

In [42]:
dialog

,appeal_id,datetime,client_id,user_id,msg_type,channel_id,channel_name,channel_type,answer_time,text
0,323874,2088-08-08 16:44:02+03,NaN,19670.0,0,73,Webdealer,Веб-чат,12.0,Привет.))
1,323870,2088-08-08 16:44:01+03,NaN,19671.0,0,73,Webdealer,Веб-чат,NaN,Все подписки отключены. Как это делать самосто...
2,323861,2088-08-08 16:44:01+03,NaN,19418.0,0,133,Нижний Новгород B2C,Веб-чат,NaN,Вы забыли пароль и хотите войти в кабинет?
3,323850,2088-08-08 16:44:01+03,NaN,19578.0,0,2,Viber,Viber,54.0,Можем отправить заявку на подключение новых но...
4,322276,2088-08-08 16:44:01+03,NaN,NaN,2,0,NaN,NaN,NaN,Обращение закрыто в связи с неактивностью поль...
5,323867,2088-08-08 16:43:58+03,569008.0,NaN,0,177,Омск B2C,Веб-чат,NaN,Тогда проще перейти с номером в мегафон Включа...
6,323868,2088-08-08 16:43:57+03,NaN,19671.0,0,195,Бурятия B2C,Веб-чат,18.0,По какой причине решили перейти?
7,323873,2088-08-08 16:43:56+03,NaN,19389.0,0,77,Краснодар B2C,Веб-чат,13.0,"Здравствуйте, меня зовут Оксана. С удовольстви..."
8,323876,2088-08-08 16:43:54+03,NaN,19752.0,12,79,Волгоград B2C,Веб-чат,NaN,Приветствуем Вас в службе поддержки клиентов! ...
9,323876,2088-08-08 16:43:54+03,NaN,NaN,4,0,NaN,NaN,NaN,NaN


In [ ]:
kf = KFold(n_splits=20)


In [32]:
ft.save('fast_text')